# Training 1M images using Inception retrained on our validated 10K images


This code is adapted from tensorflow/tensorflow/models/image/imagenet/classify_image.py 

In [1]:
import os.path
import re
import sys
import tarfile
import time
import multiprocessing as mp
import itertools

import tensorflow.python.platform
from six.moves import urllib
import numpy as np
import tensorflow as tf
import h5py
import glob
import cPickle as pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.platform import gfile

from run_inference import predict_star, predict

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Code to read in a graph + optimal parameters

In [2]:
def create_graph(pb_file):
    """"Creates a graph from saved GraphDef file and returns a Graph object.
    Returns:
    Graph holding the trained Inception network.
    """
    model_filename = pb_file
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

In [3]:
#graph = create_graph"/data/retrain_manualtags/output_graph_best.pb")
create_graph("/data/output_graphfinal.pb")
sess = tf.Session()

In [4]:
params = pickle.load(open("/data/10k_aug_outputs/output_params_lr1e-3_adam9800.pkl", 'r'))
fw = params["final_weights"]
fb = params["final_biases"]
print fb.shape
print fw.shape

(12,)
(2048, 12)


In [5]:
labels = []
with open("/data/10k_aug_outputs/output_labels9800.txt", 'r') as ifile:
    for line in ifile:
        labels.append(line.rstrip())
labels

['animals',
 'nature',
 'text',
 'maps',
 'people',
 'seals',
 'miniatures',
 'objects',
 'architecture',
 'decorations',
 'landscapes',
 'diagrams']

## Code to calculate scores on an image

In [8]:
# from http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html
def to_rgb(im):
    w, h = im.shape[:2]
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = im
    ret[:, :, 1] = im
    ret[:, :, 2] = im
    return ret

# from http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html
def to_rgbs(im):
    n, w, h = im.shape[:3]
    ret = np.empty((n, w, h, 3), dtype=np.uint8)
    ret[:, :, :, 0] = im[:,:,:,0]
    ret[:, :, :, 1] = im[:,:,:,0]
    ret[:, :, :, 2] = im[:,:,:,0]
    return ret

# from https://gist.github.com/yusugomori/4462221
def softmax(x):
    e = np.exp(x - np.max(x))  # prevent overflow
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else:  
        return e / np.array([np.sum(e, axis=1)]).T  # ndim = 2

In [ ]:
def run_inference_on_image(image, fileType, visualize=True, verbose=True):
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.

    softmax_tensor = sess.graph.get_tensor_by_name('pool_3/_reshape:0')
    
    if fileType=='jpg':
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)
        image_data = gfile.FastGFile(image).read()
        predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
    elif fileType=='arr':
        image_data = [to_rgb(image)]
        predictions = sess.run(softmax_tensor, {'ExpandDims:0': image_data})
    else:
        print "Must be jpg or arr."
        return

    preds = softmax((np.dot(predictions, fw) + fb)[0])
    top_k = preds.argsort()[-12:][::-1]
    
    for node_id in top_k:
      score = preds[node_id]
      if verbose:
            print '%s (score = %.5f)' % (labels[int(node_id)], score)
       
    if visualize:
        if fileType=='jpg': plt.imshow(mpimg.imread(image), cmap=mpl.cm.gray)
        else:
            plt.imshow(image, cmap=mpl.cm.gray)

    return preds

In [ ]:
x = mpimg.imread("/data/reorg3/Maps/003055002_10_000599_1_.jpg")
print x.shape
preds = run_inference_on_image(x, fileType="arr")

## Read in 10K images from our hdf5 file, and score their scores in a dict

In [6]:
# read in hdf5 file
image_hdf5 = h5py.File('/data/image_data.hdf5','r')
(image_metadata, book_metadata, image_to_idx) = pickle.load(open("/data/all_metadata_w11ktags.pkl", 'r'))

In [ ]:
image_to_idx.keys()[:5]

In [12]:
# choose 10k pictures not in the tagged set

num_images = 10000
images_for_tagging = list(np.random.choice(list(set(image_to_idx.keys()) - tagged), size=10000, replace=False))
images_for_tagging[:5]

NameError: name 'tagged' is not defined

In [24]:
images_to_scores = {}
args = {"images_to_scores": images_to_scores, 
        "image_to_idx": image_to_idx, 
        "image_hdf5": image_hdf5, 
        "fw": fw, 
        "fb": fb, 
        "sess": sess}

s = time.time()

for img in image_to_idx:
    try:
        predict_star((img, args))
    except:
        continue
    break
f = time.time()
print f - s

0.338056087494


In [16]:
a = image_hdf5["Chunk0"][:10]

In [17]:
b = to_rgbs(a)
b.shape

(10, 224, 224, 3)

In [10]:
softmax_tensor = sess.graph.get_tensor_by_name('pool_3:0')
s = time.time()
a = to_rgbs(image_hdf5["Chunk0"][:10])
predictions = sess.run(softmax_tensor, {'ExpandDims:0': a})[:,0,0,:]
preds = np.dot(predictions, fw) + fb
preds = np.array([softmax(preds[i]) for i in range(preds.shape[0])])
#top_k = preds.argsort()[-12:][::-1]
f = time.time()
print f - s

3.00107693672


In [49]:
np.array([softmax(preds[i]) for i in range(preds.shape[0])])

array([[  1.75737689e-04,   1.02026061e-04,   4.98217717e-03,
          4.85685587e-06,   1.02797442e-03,   5.87578535e-01,
          4.00601804e-01,   2.42072949e-03,   1.07868400e-03,
          1.38594361e-03,   5.95708960e-04,   4.58377472e-05],
       [  2.15077616e-05,   2.75007933e-05,   1.85669761e-03,
          8.34639504e-05,   3.02390486e-04,   1.90650150e-02,
          6.75299644e-01,   2.04064627e-03,   1.37578609e-04,
          3.00988913e-01,   9.23561765e-06,   1.67372084e-04],
       [  9.74729119e-07,   4.92729923e-05,   5.06105553e-03,
          1.29475484e-05,   3.84568266e-05,   4.57915328e-02,
          6.69574857e-01,   1.89486111e-03,   1.01982346e-06,
          2.77573735e-01,   9.18649789e-07,   3.11069527e-07],
       [  1.62652746e-01,   1.79588422e-03,   6.08959794e-03,
          5.86105213e-02,   3.63012403e-01,   2.35896647e-01,
          8.16231687e-03,   6.69940189e-02,   1.16148004e-02,
          8.14899504e-02,   3.26358154e-03,   4.17619915e-04],
    

In [ ]:
# let's export a image to tag mapping so we can start validating
image_to_tags = {}
for img in images_to_scores:
    image_to_tags[img] = labels[np.argmax(images_to_scores[img])]
pickle.dump(image_to_tags, open("image_to_tags_10k.pkl", 'w'))

## Extra code (to try to parallelize)

In [ ]:
# lolol i tried to parallelize
# make multiprocessing pool; lets skip this for now

p = mp.Pool(8)
manager = mp.Manager()
images_to_scores = manager.dict()
images_to_scores = {}


# p.map(real_predict_star, itertools.izip(images_for_tagging[:10], itertools.repeat(args)))
# p.close()
# p.join()

a = mp.Process(target=predict, args=((images_for_tagging[7], args)))
b = mp.Process(target=predict, args=((images_for_tagging[8], args)))
a.start()
b.start()
a.join()
b.join()

In [ ]:
images_to_scores

In [ ]:
x = [a.name for a in sess.graph.get_operations()]
for layer in x:
    print layer

In [ ]:
sess.graph.

In [ ]:
a = sess.graph
b = a.get_tensor_by_name("final_result:0")
b

In [ ]:
b